In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Load the raw data
netflix_df = pd.read_csv('/Users/vanessaokosun/Downloads/netflix_titles.csv')

## 1. Handeling Missing Values